<a href="https://colab.research.google.com/github/AliBenovaa/IANNwTF_Group24/blob/main/Homework_11_Goup_24.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

install the necessary packages

In [ ]:
!pip install -q "tensorflow-text==2.8.*"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 27.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 498.0/498.0 MB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 30.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.3/462.3 KB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 20.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 KB 1.1 MB/s eta 0:00:00


In [ ]:
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 53.9 MB/s eta 0:00:00


In [ ]:
# imports
import os
import tensorflow as tf
import re
import numpy as np
import tensorflow_text as tf_text
from collections import Counter
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt


import sentencepiece as sp



In [ ]:
#accessing google drive
from google.colab import drive
drive.mount ("/content/drive")
os.chdir ("drive/MyDrive")


Mounted at /content/drive


In [ ]:
# Load text from file
with open('bible.txt', 'r') as file:
    text = file.read()


tokens = text.lower().replace('\n', ' ').replace('[^\w\s]','').split()
print(len(tokens))

token_str = '\n'.join(tokens)

with open('bible_text.txt', 'w') as file:
    file.write(token_str)

821122


In [ ]:
import sentencepiece as sp
import tensorflow_text as tf_text

# Train tokenizer and load the model
sp.SentencePieceTrainer.train(input='bible_text.txt', model_prefix='tokenizer_model', model_type="unigram", vocab_size=2000)
trained_tokenizer_model = tf.io.gfile.GFile('tokenizer_model.model', "rb").read()

# Create tokenizer
tokenizer = tf_text.SentencepieceTokenizer(model=trained_tokenizer_model, out_type=tf.int32)

# Tokenize text
tokenized_text = tokenizer.tokenize([text]).to_list()[0]

# Define sequence length
m = 32

# Use sliding window to split into sequences of length m+1
sequences = tf_text.sliding_window(data=tokenized_text, width=m+1, axis=0)
dataset = tf.data.Dataset.from_tensor_slices(sequences)
# Define a function to split each sequence into inputs and targets
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

# Map the function to the sequences dataset
dataset = dataset.map(split_input_target)

# Shuffle the dataset
buffer_size = 10000
dataset = dataset.shuffle(buffer_size)

# Batch the dataset
batch_size = 64
dataset = dataset.batch(batch_size, drop_remainder=True)

# Print the first 5 input and target sequences
for input_example, target_example in dataset.take(10):
    print('Input data: ', input_example.numpy())
    print('Target data:', target_example.numpy())
    print()



Input data:  [[   9    0   80 ...  197   16   83]
 [  16   22   12 ...  144  872 1760]
 [   5   13  144 ...    4    9    0]
 ...
 [ 299    6   34 ...  463    7  822]
 [   0  103  195 ...  103  195   21]
 [  10  444   52 ...  278   10    5]]
Target data: [[   0   80   33 ...   16   83  188]
 [  22   12    4 ...  872 1760    6]
 [  13  144   19 ...    9    0   13]
 ...
 [   6   34    4 ...    7  822   71]
 [ 103  195   21 ...  195   21  344]
 [ 444   52    9 ...   10    5   14]]

Input data:  [[   9    0   66 ...   49  158    9]
 [ 124    4  127 ...  159   66    8]
 [ 188  230   18 ...  197   51    9]
 ...
 [  26    3    5 ...  484 1411   26]
 [ 109   26   69 ...  872  276    7]
 [  27   35  746 ...    5  710    9]]
Target data: [[   0   66   33 ...  158    9    0]
 [   4  127    6 ...   66    8   66]
 [ 230   18   69 ...   51    9    0]
 ...
 [   3    5   74 ... 1411   26 1180]
 [  26   69    6 ...  276    7  525]
 [  35  746   80 ...  710    9  105]]

Input data:  [[  43  946   11 ... 

In [ ]:
class TokenAndPositionEmbedding(tf.keras.layers.Layer):
    def __init__(self, max_seq_length, vocab_size, embed_dim):
        super(TokenAndPositionEmbedding, self).__init__()
        self.token_emb = tf.keras.layers.Embedding(input_dim=vocab_size, output_dim=embed_dim)
        self.pos_emb = tf.keras.layers.Embedding(input_dim=max_seq_length, output_dim=embed_dim)
        
    def call(self, x):
        seq_length = tf.shape(x)[-1]
        positions = tf.range(start=0, limit=seq_length, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x + positions

In [ ]:
max_seq_length = m
vocab_size = 2000
embed_dim = 128

model = tf.keras.Sequential([
    TokenAndPositionEmbedding(max_seq_length, vocab_size, embed_dim),
    # Add additional layers as needed
])

In [ ]:
class TransformerBlock(tf.keras.layers.Layer):
    def __init__(self, num_heads, key_dim, dense_units, dropout_rate=0.1):
        super(TransformerBlock, self).__init__()
        self.mha = tf.keras.layers.MultiHeadAttention(num_heads=num_heads, key_dim=key_dim)
        self.dense1 = tf.keras.layers.Dense(units=dense_units, activation='relu')
        self.dense2 = tf.keras.layers.Dense(units=key_dim, activation=None)
        self.dropout1 = tf.keras.layers.Dropout(rate=dropout_rate)
        self.dropout2 = tf.keras.layers.Dropout(rate=dropout_rate)
        self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
    
    def call(self, inputs, training=True):
        # Multi-Head Attention block
        mha_out = self.mha(inputs, inputs, use_scale=True, attention_mask=None, training=training)
        mha_out = self.dropout1(mha_out, training=training)
        mha_out = self.layernorm1(inputs + mha_out)
        
        # Feed-forward neural network
        ffn_out = self.dense1(mha_out)
        ffn_out = self.dense2(ffn_out)
        ffn_out = self.dropout2(ffn_out, training=training)
        ffn_out = self.layernorm2(mha_out + ffn_out)
        
        return ffn_out


In [ ]:
class GPT(tf.keras.Model):
    def __init__(self, vocab_size, num_layers, d_model, num_heads, dff, maximum_position_encoding, dropout_rate=0.1):
        super(GPT, self).__init__()
        
        self.vocab_size = vocab_size
        self.d_model = d_model
        self.num_layers = num_layers
        
        self.embedding = tf.keras.layers.Embedding(input_dim=self.vocab_size, output_dim=self.d_model)
        self.positional_encoding = self.positional_encoding(maximum_position_encoding, self.d_model)
        self.dropout = tf.keras.layers.Dropout(rate=dropout_rate)
        self.encoder_layers = [TransformerBlock(d_model=self.d_model, num_heads=num_heads, dff=dff, rate=dropout_rate) for _ in range(num_layers)]
        self.final_layer = tf.keras.layers.Dense(units=self.vocab_size)
        
        self.optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
        self.loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
        self.train_loss = tf.keras.metrics.Mean(name='train_loss')
        self.train_accuracy = tf.keras.metrics.Mean(name='train_accuracy')
    
    def positional_encoding(self, position, d_model):
        angle_rads = self.get_angles(np.arange(position)[:, np.newaxis], np.arange(d_model)[np.newaxis, :], d_model)
        sines = np.sin(angle_rads[:, 0::2])
        cosines = np.cos(angle_rads[:, 1::2])
        pos_encoding = np.concatenate([sines, cosines], axis=-1)
        pos_encoding = pos_encoding[np.newaxis, ...]
        return tf.cast(pos_encoding, dtype=tf.float32)
        
    def get_angles(self, pos, i, d_model):
        angle_rates = 1 / np.power(10000, (2 * (i//2)) / np.float32(d_model))
        return pos * angle_rates
    
    def call(self, inputs):
        seq_length = tf.shape(inputs)[1]
        
        # adding embedding and position encoding
        x = self.embedding(inputs)
        x *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))
        x += self.positional_encoding[:, :seq_length, :]
        x = self.dropout(x)
        
        # running through the encoder layers
        for i in range(self.num_layers):
            x = self.encoder_layers[i](x)
        
        # passing through the final layer
        output = self.final_layer(x)
        return output
    
    def reset_metrics(self):
        self.train_loss.reset_states()
        self.train_accuracy.reset_states()
    
    def train_step(self, inputs):
        inputs, targets = inputs
        
        with tf.GradientTape() as tape:
            predictions = self(inputs, training=True)
            loss = self.loss_object(targets, predictions)
        gradients = tape.gradient(loss, self.trainable_variables)
        self.optimizer.apply_gradients(zip(gradients, self.trainable_variables))
        self.train_loss(loss)
        self.train_accuracy(targets, predictions)
      
    def generate_text(self, prompt, output_length, top_k, temperature):
        input_ids = self.tokenizer.encode(prompt)
        input_ids = tf.expand_dims(input_ids, 0)

        for _ in range(output_length):
            logits = self(input_ids)
            last_logits = logits[:, -1, :]
            last_logits /= temperature
            top_k_logits, top_k_indices = tf.math.top_k(last_logits, k=top_k, sorted=True)

            top_k_probs = tf.nn.softmax(top_k_logits)
            sampled_token_index = tf.random.categorical(top_k_probs, num_samples=1)[-1, 0].numpy()
            sampled_token = top_k_indices[0][sampled_token_index].numpy()
            
            input_ids = tf.concat([input_ids, tf.expand_dims([sampled_token], 0)], axis=-1)

        text = self.tokenizer.decode(input_ids[0].numpy())
        return text
    """def generate_text(self, prompt, output_length, top_k, temperature):
        prompt = tf.constant(self.tokenizer.encode(prompt))[tf.newaxis, :]
        
        for i in range(output_length):
            predictions = self(prompt)
            last_token_logits = predictions[:, -1, :]
            last_token_logits = last_token_logits / temperature
            sorted_logits, sorted_indices = tf.math.top_k(last_token_logits, k=top_k)
            sorted_indices = tf.squeeze(sorted_indices)
            next_token = tf
"""

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 93.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 23.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 99.6 MB/s eta 0:00:00


In [ ]:
import torch
import torch.nn.functional as F
from torch.utils.tensorboard import SummaryWriter
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# Load the model and tokenizer
model = GPT2LMHeadModel.from_pretrained('gpt2')
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

# Load the text data and split it into training and validation sets
with open('bible_text.txt', 'r', encoding='utf-8') as f:
    text = f.read()
train_text = text[:int(0.9*len(text))]
val_text = text[int(0.9*len(text)):]

# Set up Tensorboard logging
log_dir = 'logs'
writer = SummaryWriter(log_dir)

# Define the generation method
def generate_text(prompt, length):
    input_ids = tokenizer.encode(prompt, return_tensors='pt')
    output = model.generate(input_ids, max_length=length, do_sample=True, top_p=0.95, top_k=60)
    return tokenizer.decode(output[0], skip_special_tokens=True)

# Train the model
batch_size = 4
epochs = 100
steps_per_epoch = len(train_text) // (batch_size * 512)

optimizer = torch.optim.Adam(model.parameters())

for epoch in range(epochs):
    print(f'Epoch {epoch}')
    for step in range(steps_per_epoch):
        # Sample a random chunk of text for this batch
        start = step * batch_size * 512
        end = (step + 1) * batch_size * 512
        input_text = train_text[start:end]

        # Split the input text into smaller chunks to handle sequence length error
        chunks = [input_text[i:i+1024] for i in range(0, len(input_text), 1024)]
        #chunks = [input_text[i:i+5] for i in range(0, 5, 5)]
        # Encode the input text chunks and pass them through the model
        for chunk in chunks:
            input_ids = tokenizer.encode(chunk, return_tensors='pt')
            labels = input_ids.clone()
            outputs = model(input_ids, labels=labels)
            loss = outputs.loss

            # Backpropagate and update the model parameters
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()

            # Log the loss to Tensorboard
            writer.add_scalar('training loss', loss.item(), epoch * steps_per_epoch + step)

    # Generate some sample text and log it to Tensorboard
    prompt = "What is"
    generated_text = generate_text(prompt, 200)
    writer.add_text('generated text', generated_text, epoch)

    # Evaluate the model on the validation set and log the loss to Tensorboard
    val_input_ids = tokenizer.encode(val_text, return_tensors='pt')
    val_labels = val_input_ids.clone()
    val_outputs = model(val_input_ids, labels=val_labels)
    val_loss = val_outputs.loss
    writer.add_scalar('validation loss', val_loss.item(), epoch)

    # Save the model weights
    model_path = f'models/model_{epoch}.pt'
    torch.save(model.state_dict(), model_path)


Epoch 0
